In [1]:
from setup_env import setup_environment

setup_environment()

In [2]:
import glob
import os

import pandas as pd

from project.config import HIDDEN_CELL

In [3]:
folder_path = "data/history/raw"
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

dfs = []
for file in csv_files:
    df = pd.read_csv(file)

    is_walkable_cols = [col for col in df.columns if col.endswith(".is_walkable")]

    indices = []
    for col in is_walkable_cols:
        if col.startswith("choosen_pattern"):
            continue
        try:
            idx = int(col.split(".")[0])
            indices.append(idx)
        except ValueError:
            continue

    if indices:
        indices.sort()
        middle_idx = indices[len(indices) // 2]
    else:
        middle_idx = None

    selected_columns = ["success", "choosen_pattern.is_walkable"]

    for col in is_walkable_cols:
        if col.startswith("choosen_pattern"):
            continue

        try:
            col_idx = int(col.split(".")[0])
            if col_idx != middle_idx:
                selected_columns.append(col)
        except ValueError:
            selected_columns.append(col)

    df_selected = df[selected_columns]
    dfs.append(df_selected)

df = pd.concat(dfs, ignore_index=True)
df = df.fillna(HIDDEN_CELL)
df = df.drop_duplicates()

print(f"Successfully concatenated {len(csv_files)} files")
print(f"Final dataframe shape: {df.shape}")
print("Columns:, df.columns.tolist()")

Successfully concatenated 13732 files
Final dataframe shape: (560, 10)
Columns:, df.columns.tolist()


In [4]:
df.to_csv("data/history/concat/history_3x3-500.csv", index=False)